In [1]:
import os
import time
from qick import *
from pynq import Overlay, DefaultIP, PL

from qickquack import *

In [2]:
def bitfile_path():
    """Choose the default firmware path for this board.

    Parameters
    ----------

    Returns
    -------
    str
        absolute path to the firmware bitfile distributed with the QICK library
    """
    #board2file =  {'ZCU216' :'qick_216.bit','ZCU111' :'qick_111.bit','RFSoC4x2' :'qick_4x2.bit'}
    #filename = board2file[os.environ['BOARD']]
    filename = "axi_pvp_gen_v7_3.0.bit"
    #src = os.path.join(os.path.dirname(qick.__file__), filename)
    src = os.path.join("/home/xilinx/bit_files/axi_pvp_gen_v7_3.0", filename)
    return src

In [3]:
class TestSoc(Overlay):
    def __init__(self, bitfile=None, **kwargs):
        if bitfile is None:
            Overlay.__init__(self, bitfile_path(), **kwargs)
        else:
            Overlay.__init__(self, bitfile, **kwargs)
        

In [257]:
class AxiPvpGen(SocIp):
    """Control the axi_pvp_gen_v7_x IP"""
    
    # PVP Gen Control Registers
    
    # START_VAL_0_REG : 20 bit
    # START_VAL_1_REG : 20 bit
    # START_VAL_2_REG : 20 bit
    # START_VAL_3_REG : 20 bit
    
    # STEP_SIZE_0_REG : 20 bit
    # STEP_SIZE_1_REG : 20 bit
    # STEP_SIZE_2_REG : 20 bit
    # STEP_SIZE_3_REG : 20 bit
    
    # DEMUX_0_REG : 5 bit 
    # DEMUX_1_REG : 5 bit
    # DEMUX_2_REG : 5 bit 
    # DEMUX_3_REG : 5 bit 
    
    # DAC_0_GROUP_REG: 2 bit
    # DAC_1_GROUP_REG: 2 bit
    # DAC_2_GROUP_REG: 2 bit
    # DAC_3_GROUP_REG: 2 bit
    
    # CTRL_REG: 4 bit
    # MODE_REG: 2 bit
    # CONFIG_REG: 29 bit
    
    # DWELL_CYCLES_REG = 32 bit
    # CYCLES_TILL_READOUT_REG = 16 bit
    # PVP_WIDTH_REG: 10 bit
    # NUM_DIMS_REG: 3 bit
    # TRIGGER_USER_REG: 1 bit
    
    
    bindto = ['user.org:user:axi_pvp_gen_v7:4.0']
    
    def __init__(self, description, **kwargs):
        super().__init__(description)
        
        #map register names to offsets
        self.REGISTERS = {
            'START_VAL_0_REG':0, 
            'START_VAL_1_REG':1,
            'START_VAL_2_REG':2,
            'START_VAL_3_REG':3,
            
            'STEP_SIZE_0_REG':4,
            'STEP_SIZE_1_REG':5,
            'STEP_SIZE_2_REG':6,
            'STEP_SIZE_3_REG':7,
            
            'DEMUX_0_REG': 8,
            'DEMUX_1_REG': 9,
            'DEMUX_2_REG': 10,
            'DEMUX_3_REG': 11,
            
            'DAC_0_GROUP_REG': 12,
            'DAC_1_GROUP_REG': 13,
            'DAC_2_GROUP_REG': 14,
            'DAC_3_GROUP_REG': 15,
            
            'CTRL_REG': 16,
            'MODE_REG': 17,
            'CONFIG_REG': 18,
            
            'DWELL_CYCLES_REG': 19,
            'CYCLES_TILL_READOUT_REG': 20,
            'PVP_WIDTH_REG': 21,
            'NUM_DIMS_REG': 22,
            
            'TRIGGER_USER_REG': 23
            
        }
        
        #default register values
        
        self.START_VAL_0_REG = 0
        self.START_VAL_1_REG = 0
        self.START_VAL_2_REG = 0
        self.START_VAL_3_REG = 0
        
        self.STEP_SIZE_0_REG = 0
        self.STEP_SIZE_1_REG = 0
        self.STEP_SIZE_2_REG = 0
        self.STEP_SIZE_3_REG = 0
        
        self.DEMUX_0_REG = 0 # if we don't set these, expect weird behavior on dac 0 when it tries to do it all
        self.DEMUX_1_REG = 0
        self.DEMUX_2_REG = 0
        self.DEMUX_3_REG = 0
        
        self.DAC_0_GROUP_REG = 0 # by default, assign one DAC per group
        self.DAC_1_GROUP_REG = 1
        self.DAC_2_GROUP_REG = 2
        self.DAC_3_GROUP_REG = 3
        
        
        ## set up to be [ ldac_user, clrn_user, rstn_user, trigger_from ]
        ##    if you are in mode 3, you are able to control these registers. If not, changing these won't effect anthing.
        ##    trigger from tells the FSM whether it should be receiving
        ##         the signal to make the pvp plot from the PMOD pins (0)
        ##         or from the user in jupyter notebook (1). In both cases,
        ##         the system will wait until the "trigger" has gone up again before allowing
        ##         the user to run the net step of the pvp plot - it stops itself in stall
        ##         state until the flag is raised again. Especially because it is possible to
        ##         accidentally loop all the way back around into the stall state (i.e. accidentally starting
        ##         another pvp plot immediately after the first, I would recommend
        ##         tracking the number of steps you have taken.
        self.CTRL_REG = 14
        
        
        self.MODE_REG = 0
        self.CONFIG_REG = 0
        
        self.DWELL_CYCLES_REG = 38400 # at board speed of 384 MHz, 38400 dwell cycles is 100 us
        self.CYCLES_TILL_READOUT = 10
        self.PVP_WIDTH_REG = 256
        self.NUM_DIMS_REG = 0
        
        self.TRIGGER_USER_REG = 0 #default is 0 for trigger coming from qick, set for 1 for triggering manually for tests
        
        

    # ################################
    # Methods
    # ################################
   
        
#     def check_lock(self, registerName = "<name of locked register>"):
#         if (self.CTRL_REG & 0b1 == 1):
#             raise RuntimeError (registerName + " cannot be changed while pvp plot is running.")
            
            
            
    ## Setters
    
    # many axis setters
    
    def set_any_axis(self, axis='', axis_reg_dict={}, val=0):
        '''helper method for any method that has four available axes'''
        
        if axis in axis_reg_dict:
            reg_str = axis_reg_dict[axis]
            setattr(self, reg_str, val)

        else:
            raise ValueError("No valid axis was specified. Valid axis arguments are '0', '1', '2', '3'")
    
    def set_start(self, axis = '', start_val = 0):
        '''method to set start val 
            (note that we want a method for this because we don't want to worry about registers outside this class)'''
        start_regs = {'0': 'START_VAL_0_REG', '1': 'START_VAL_1_REG', '2': 'START_VAL_2_REG', '3': 'START_VAL_3_REG'}
#         self.check_lock("Start values")
        self.set_any_axis(axis = axis, axis_reg_dict = start_regs, val = start_val)
    
    def set_step_size(self, axis = '', step_size = 0):
        '''sets size of step (in Volts)'''
        step_size_regs = {'0': 'STEP_SIZE_0_REG', '1': 'STEP_SIZE_1_REG', '2': 'STEP_SIZE_2_REG', '3': 'STEP_SIZE_3_REG'}
#         self.check_lock("step size")
        self.set_any_axis(axis = axis, axis_reg_dict = step_size_regs, val = step_size)
            
    def set_demux(self, axis = '', demux = 0):
        """Set demux value for a given axis"""
#         self.check_lock("Demux values")
        demux_regs = {'0': 'DEMUX_0_REG', '1': 'DEMUX_1_REG', '2': 'DEMUX_2_REG', '3': 'DEMUX_3_REG'}
        
        #note to self: do we specify demux value or ask for board num and dac num?
        if (demux >= 0 and demux < 32):
            self.set_any_axis(axis = axis, axis_reg_dict = demux_regs, val = demux)
        else:
            raise ValueError("Demux value must be in the range 0-31 inclusive")
    
    def set_group(self, axis = '', group = 0):
        '''Set with which group a particular DAC should update'''
        group_regs = {'0': 'DAC_0_GROUP_REG', '1': 'DAC_1_GROUP_REG', '2': 'DAC_2_GROUP_REG', '3': 'DAC_3_GROUP_REG'}
#         self.check_lock("groups")
        self.set_any_axis(axis = axis, axis_reg_dict = group_regs, val = group)
        
        
    #the next five are ctrl reg manipulating methods
    
    def set_ldac(self, ldac = 1):
        """Toggle the value of the LDAC pin, if in mode 3"""
        #check if mode allows for manual control
        if (self.MODE_REG == 3):
            #clear bit and set it
            self.CTRL_REG &= 0b0111
            print(self.CTRL_REG)
            self.CTRL_REG |= (ldac << 3)
            print(self.CTRL_REG)
        else:
            print("wrong mode (need to be in mode 3 to change ldac manually)")
            
    def set_clr(self, clr = 1):
        """Clear all DACs via CLRN pin"""
        #WARNING THIS WILL  NOT STOP YOU FROM CLEARING EVEN IN THE MIDDLE OF A PVP PLOT
        self.CTRL_REG &= 0b1011
        self.CTRL_REG |= (clr << 2)
        
    def set_reset(self, resetn = 1):
        """Reset all DACs via RSTN pin"""
        #WARNING THIS WILL  NOT STOP YOU FROM RESETTING EVEN IN THE MIDDLE OF A PVP PLOT
        self.CTRL_REG &= 0b1101
        self.CTRL_REG |= (resetn << 1)
        
    def set_trigger_settings(self, trigger_set=0):
        """Set up whether you're controlling pvp via PMOD or user"""
        self.CTRL_REG &= 0b1110
        self.CTRL_REG |= (trigger_set << 0)
        
        
        
    def set_user_trigger(self, user_trig=0):
        """Set the trigger that's sent into the user bits of this system. It isn't read unless CTRL bit 0 is 1 """
        self.TRIGGER_USER_REG &= 0
        self.TRIGGER_USER_REG |= user_trig
        
        
        
    def get_trig(self):
        return (self.CTRL_REG & 0b0001)
    # regular registers
            
    def set_dwell_cycles(self, dwell_cycles = 38400):
        """Set number of clock cycles in between each step"""
#         self.check_lock("Dwell cycles")
        if (dwell_cycles < 1250):
            raise ValueError("Dwell cycles must be at least 1250 so that all SPI messages can send")
        self.DWELL_CYCLES_REG = dwell_cycles
        
    def set_readout_cycles(self, cycles_till = 400):
        """Set number of cycles during which the measurement may be read out"""
#         self.check_lock("Readout cycles")
        self.CYCLES_TILL_READOUT = cycles_till
    
        
    def set_pvp_width(self, pvp_width = 256): #this default value is so if someone accidentally runs the method without a argument, the new value is just the default reset value
        """Set the width in pixels of a pvp"""
#         self.check_lock("Pvp width")
        self.PVP_WIDTH_REG = pvp_width
        
    def set_num_dims(self, num_dims = 0):
        """Set the number of groups looped through in the pvp plot"""
#         self.check_lock("Number of dimensions")
        self.NUM_DIMS_REG = num_dims
                
    def set_mode(self, m = 0):
        """Set operation mode of the pvp gen block"""
#         self.check_lock("Mode")
        if (m < 0 or m > 3):
            raise ValueError("Mode must be 0b00, 0b01, 0b10, or 0b11.")
        self.MODE_REG = m
        
    # we don't ever just set the config reg so it's not in the simple setters- see next section
    
    ## Compound methods
            
    def report_settings(self):
        """Report all pvp gen registers' current value"""
        print("Start of DAC 0: ", hex(self.START_VAL_0_REG))
        print("Start of DAC 1: ", hex(self.START_VAL_1_REG))
        print("Start of DAC 2: ", hex(self.START_VAL_2_REG))
        print("Start of DAC 3: ", hex(self.START_VAL_3_REG))
        
        print("Step Size DAC 0: ", hex(self.STEP_SIZE_0_REG))
        print("Step Size DAC 1: ", hex(self.STEP_SIZE_1_REG))
        print("Step Size DAC 2: ", hex(self.STEP_SIZE_2_REG))
        print("Step Size DAC 3: ", hex(self.STEP_SIZE_3_REG))
        
        print("DEMUX 0: ", hex(self.DEMUX_0_REG))
        print("DEMUX 1: ", hex(self.DEMUX_1_REG))
        print("DEMUX 2: ", hex(self.DEMUX_2_REG))
        print("DEMUX 3: ", hex(self.DEMUX_3_REG))
        
        print("Control Reg: ", hex(self.CTRL_REG))
        print("Mode Reg", hex(self.MODE_REG))
        print("Arbitrary 24 bits of SPI: ", hex(self.CONFIG_REG))
        
        print("Number of Dwell Cycles: ", hex(self.DWELL_CYCLES_REG))
        print("Cycles till Trigger AWGs: ", hex(self.CYCLES_TILL_READOUT))
        print("Size of PVP plot (square): ", hex(self.PVP_WIDTH_REG))
        print("Number of DACs Running: ", hex(self.NUM_DIMS_REG))
        print("Trigger source: ", "user" if (self.TRIGGER_USER_REG) else "qick_processor")
       
    
    ## Defining a Function to help run a fast PvP plot
    ## it also assumes you've arleady run "setup pvp" for yourself so that everything lines up

    def run_pvp_demo(self):
        for i in range ((dac_call.PVP_WIDTH_REG)**(dac_call.NUM_DIMS_REG)):
            dac_call.set_user_trigger(1)
            time.sleep(0.01)
            dac_call.set_user_trigger(0)
            time.sleep(0.2)
            
    def self_pvp(self):
        dac_call.set_user_trigger(1)
        time.sleep(0.01)
        dac_call.set_user_trigger(0)
        time.sleep(0.05)
        
    def send_arbitrary_SPI(self, demux_int = 0b00000, reg = 0b0000, data_int = 0x00000):
        '''Allow the user to specify an arbitrary dac (demux_int) and send it an arbitrary 24 bit message (data_int)
           Raises the done flag when finished and cannot be run again until pvp trigger reg is cleared'''
        
#         self.check_lock("Arbitrary spi")
        
        demux_shift = demux_int << 24
        reg_shift = reg << 20
        out = demux_shift + reg_shift + data_int
        print("Writing config reg to " + str(bin(out)))
        self.CONFIG_REG = out
        time.sleep(0.1)
        self.CONFIG_REG = 0
        
    def setup_pvp(self, cfg = {'startvals': [0,0,0,0],
                              'stepsizes': [0,0,0,0],
                              'demuxvals': [0,0,0,0],
                                'groups': [0,1,2,3],
                                 'mode': 0,
                               'width': 16,
                               'num_dims': 4
                              }
                 ):
        '''sets up EVERYTHING for a pvp plot
        assuming a user sets everything they want here, they should only need to run this + start_pvp()'''
        
        for dac in range (len(cfg['startvals'])):
            self.set_start(axis = str(dac), start_val = volt2reg(cfg['startvals'][dac]))
            self.set_step_size(axis = str(dac), step_size = volt2reg(cfg['stepsizes'][dac]))
            self.set_demux(axis = str(dac), demux = cfg['demuxvals'][dac])
            self.set_group(axis = str(dac), group = cfg['groups'][dac])
        self.set_mode(cfg['mode'])
        self.set_pvp_width(cfg['width'])
        self.set_num_dims(cfg['num_dims'])


In [258]:
## these need a class so they don't have to be updated when we change the ip block revision eg from axi_pvp_gen_v5 to v7

def init_DAC(demux = 0):
    soc.axi_pvp_gen_v7_0.send_arbitrary_SPI(demux_int = demux, reg = 0b0010, data_int = 0b0000_0000_0000_0011_0010)
    
def volt2reg(volt = 0.0):
    VREFN = 0.0
    VREFP = 5.0
    bit_res = 20

    if volt < VREFN:
        print("%s: %d V out of range." % (self.__class__.__name__, volt))
        return -1
    elif volt > VREFP:
        print("%s: %d V out of range." % (self.__class__.__name__, volt))
        return -1
    else:
        Df = (2**bit_res - 1)*(volt - VREFN)/(VREFP - VREFN)
        print("Df is " + str(bin(int(Df))))
        return int(Df)

def set_DAC(demux = 0, volts = 0.0):
    val = volt2reg(volts)
    #5 bits of demux, 4 bits of reg, 20 bits of data
    soc.axi_pvp_gen_v7_0.send_arbitrary_SPI(demux, 0b0001, val)

In [310]:
soc = TestSoc()
soc.ip_dict.keys()

dict_keys(['axi_pvp_gen_v7_0', 'zynq_ultra_ps_e_0'])

In [311]:
dac_call = soc.axi_pvp_gen_v7_0

This block has all the settings for a pvp test:

In [335]:
init_DAC(27)
set_DAC(27, 2.35)

Writing config reg to 0b11011001000000000000000110010
Df is 0b1111000010100011110
Writing config reg to 0b11011000101111000010100011110


In [331]:
## Setting the initial values as defined in my own test bench (zoe)

dac_call.set_start('0', volt2reg(5))
dac_call.set_start('1', volt2reg(4))

dac_call.set_step_size('0', volt2reg(0.3))
dac_call.set_step_size('1', volt2reg(0.2))

dac_call.set_demux('0', 2)
dac_call.set_demux('1', 4)

dac_call.set_group('0', 1)
dac_call.set_group('1', 0)

dac_call.set_dwell_cycles(2500)
dac_call.set_readout_cycles(10)

dac_call.set_num_dims(2) ## this is really important, because otherwise all the logic can and will break
dac_call.set_pvp_width(5)

dac_call.set_trigger_settings(1) # this makes it so that we have control. The system will now look to the 
                                  #    TRIGGER_USER_REG instead of the pmod_control
    
init_DAC(2)
init_DAC(4)

set_DAC(2, 2.237)
set_DAC(4, 4)

Df is 0b11111111111111111111
Df is 0b11001100110011001100
Df is 0b1111010111000010
Df is 0b1010001111010111
Writing config reg to 0b10001000000000000000110010
Writing config reg to 0b100001000000000000000110010
Df is 0b1110010100010001100
Writing config reg to 0b10000101110010100010001100
Df is 0b11001100110011001100
Writing config reg to 0b100000111001100110011001100


In [332]:
dac_call.report_settings()
i = 0

Start of DAC 0:  0xfffff
Start of DAC 1:  0xccccc
Start of DAC 2:  0x0
Start of DAC 3:  0x0
Step Size DAC 0:  0xf5c2
Step Size DAC 1:  0xa3d7
Step Size DAC 2:  0x0
Step Size DAC 3:  0x0
DEMUX 0:  0x2
DEMUX 1:  0x4
DEMUX 2:  0x0
DEMUX 3:  0x0
Control Reg:  0xf
Mode Reg 0x0
Arbitrary 24 bits of SPI:  0x0
Number of Dwell Cycles:  0x9c4
Cycles till Trigger AWGs:  0xa
Size of PVP plot (square):  0x5
Number of DACs Running:  0x2
Trigger source:  qick_processor


In [268]:
for i in range(2**(dac_call.PVP_WIDTH_REG)):
    dac_call.self_pvp()



Previous: 0b10000010101011110011011110
Now:      0b1000001000010101011110011011110
newer:    0b10000110101011110011011110

In [101]:
# LOOK AT ME!!!!!
# run this cell to config and set a DAC 
# this is the shorter (preferred) way to do the above two cells
curr_dac = 20
init_DAC(curr_dac)
set_DAC(curr_dac, 2.237)

Writing config reg to 0b10100001000000000000000110010
Df is 0b1110010100010001100
Writing config reg to 0b10100000101110010100010001100


In [102]:
soc.axi_pvp_gen_v7_0.set_trigger_source('user')
soc.axi_pvp_gen_v7_0.TRIGGER_USER_REG

AttributeError: 'AxiPvpGen' object has no attribute 'set_trigger_source'

In [103]:
for r in range (4):
    soc.axi_pvp_gen_v7_0.start_pvp()

AttributeError: 'AxiPvpGen' object has no attribute 'start_pvp'

In [ ]:
print(soc.axi_pvp_gen_v7_0.TRIGGER_USER_REG)
print(soc.axi_pvp_gen_v7_0.get_trig())
print(bin(soc.axi_pvp_gen_v7_0.CTRL_REG))

In [ ]:
soc.axi_pvp_gen_v7_0.end_pvp()

In [ ]:
soc.axi_pvp_gen_v7_0.start_pvp()